In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

In [2]:
class BNB():

    _cVec = None
    _features = None
    _targets = None
    _TS = None
    _M = None
    _indicators = None
    _theta_one = None
    _theta_zero = None
    _theta_j_one = []
    _theta_j_zero = []
    
    def __init__(self,features,targets):
        #Expecting csr_matrix
        ##Expecting features and targets (Targets are single dimension array)
        self._cVec = CountVectorizer(binary=True,min_df=0.01) ##TODO ADD min_df, max_df,etc...
        self._targets = targets
        self._TS = targets.shape[0]
        self._numO = (self._targets == 1).sum()
        self._numZ = (self._targets == 0).sum()
        self._theta_one = (self._targets == 1).sum()/targets.shape[0]
        self._theta_zero = (self._targets == 0).sum()/targets.shape[0]
        self._indicators = self._cVec.fit_transform(features)
        self._features = self._cVec.get_feature_names()
        self._M = len(self._features)
        ##TODO: Try looping over the indicators individually and from there use the self._target to check if suck value shall be
        #updated in the corrosponding theta
        for j in range(self._M):
            self._theta_j_one.append(self._indicators.getcol(j).toarray()[self._targets.to_numpy() == 1].sum()/self._numO)
            self._theta_j_zero.append(self._indicators.getcol(j).toarray()[self._targets.to_numpy() == 0].sum()/self._numZ)
            
##Creating BNB
data = pd.read_csv('train.csv', sep=',',header=0)
data = data.replace('negative',0).replace('positive',1)
feat = data['review']
targ = data['sentiment']

test = BNB(feat,targ)